# Binary Classification with Amazon Machine Learning (Learning from Disaster)

***
Copyright [2017]-[2017] Amazon.com, Inc. or its affiliates. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. A copy of the License is located at

http://aws.amazon.com/apache2.0/

or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
***

Description:  
This demo is based on the popular ["Titanic: Machine Learning from Disaster"](https://www.kaggle.com/c/titanic) Kaggle competition. I highly recommend creating an account and taking the full challenge as described on the competition page. 

From the Kaggle website:  
<i>The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.</i>

We are using the sample dataset provided of passengers on board the ship to training a model with Amazon Machine Learning to predict survival chances with the real time predictions endpoint.

Helpful resources:
[Amazon Machine Learning – Make Data-Driven Decisions at Scale](https://aws.amazon.com/blogs/aws/amazon-machine-learning-make-data-driven-decisions-at-scale/)  
[Building a Binary Classification Model with Amazon Machine Learning and Amazon Redshift](https://blogs.aws.amazon.com/bigdata/post/TxGVITXN9DT5V6/Building-a-Binary-Classification-Model-with-Amazon-Machine-Learning-and-Amazon-R)

Comments:
This is meant to be an asynchronous demo where one quickly walks through the deployment sections, runs all cells above the real time prediction section and gets back to the results after about 20 minutes.

Estimated cost: The <b>maximum</b> cost for a single run would be 0.42$. The Data Analysis and Model Building cost depends on the size of the input data, the number of attributes within it, and the number and types of transformations applied. As this is very small scale demo the cost should be even lower.
The cost of real time predictions is be negligible.

** Prerequisites: **

The user or role that executes the commands must have permissions in AWS Identity and Access Management (IAM) to perform those actions. AWS provides a set of managed policies that help you get started quickly. For our example, you need to apply the following minimum managed policies to your user or role:

* AmazonMachineLearningFullAccess 
* AmazonS3FullAccess 

Be aware that we recommend you follow AWS IAM best practices for production implementations, which is out of scope fof this workshop.


Demo Author: Stas Vonholsky

In [ ]:
import random, os, string, urllib, json, time, sys
from boto3.session import Session

# Generate session prefix
demo_pre = "ipydemo-"
N = 4
random_pre = ''.join(random.SystemRandom().choice(string.ascii_lowercase) for _ in range(N)) + "-"
resource_pre = demo_pre + random_pre
print ("\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print ("Demo session ID: " + resource_pre[:-1])
print ("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

# Fetch relevant metadata
work_dir = "resources/"
meta_data_pre = "http://169.254.169.254/latest/meta-data/"

region = 'eu-west-1'
    
session = Session(region_name=region)
s3 = session.client('s3')
ml = session.client('machinelearning')

#File paths
train_data = "train.csv"

In [ ]:
#Create bucket and upload training data
s3_bucket = resource_pre + 'aml-bucket'
s3.create_bucket(Bucket=s3_bucket,CreateBucketConfiguration={'LocationConstraint': region},)
print ("Created S3 bucket: " + s3_bucket)

path_to_train_data = work_dir + train_data
!aws s3 cp $path_to_train_data s3://$s3_bucket

#Grant read permissions to AML on Bucket
s3_bucket_policy = """{
  "Version": "2012-10-17",
  
  "Statement": [
    {
      "Sid": "AmazonML_s3:ListBucket",
      "Effect": "Allow",
      "Principal": {
        "Service": "machinelearning.amazonaws.com"
      },
      "Action": "s3:ListBucket",
      "Resource": "arn:aws:s3:::"""+ s3_bucket +""""
    },
    {
      "Sid": "AmazonML_s3:GetObject:PutObject",
      "Effect": "Allow",
      "Principal": {
        "Service": "machinelearning.amazonaws.com"
      },
      "Action": ["s3:GetObject","s3:PutObject"],
      "Resource": "arn:aws:s3:::"""+ s3_bucket +"""/*"
    }
  ]
}"""
trust_policy = json.loads(s3_bucket_policy)
trust_policy = json.dumps(s3_bucket_policy)   
    
response = s3.put_bucket_policy(
    Bucket = s3_bucket,
    Policy = s3_bucket_policy
)

print ("Bucket policy set")

In [ ]:
#Sample of the data:
sample = """
PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S"""
    
data_schema = """{
  "version": "1.0",
  "recordAnnotationFieldName": "PassengerId",
  "targetFieldName": "Survived",
  "dataFormat": "CSV",
  "dataFileContainsHeader": true,
  "attributes": [ 
    { "fieldName": "PassengerId", "fieldType": "CATEGORICAL" },
    { "fieldName": "Survived",    "fieldType": "BINARY"},
    { "fieldName": "Pclass",      "fieldType": "CATEGORICAL"},
    { "fieldName": "Name",        "fieldType": "TEXT"},
    { "fieldName": "Sex",         "fieldType": "CATEGORICAL" },
    { "fieldName": "Age",         "fieldType": "NUMERIC"},
    { "fieldName": "SibSp",       "fieldType": "NUMERIC" },
    { "fieldName": "Parch",       "fieldType": "NUMERIC" },    
    { "fieldName": "Ticket",      "fieldType": "TEXT"},
    { "fieldName": "Fare",        "fieldType": "NUMERIC"},
    { "fieldName": "Cabin",       "fieldType": "CATEGORICAL"},
    { "fieldName": "Embarked",    "fieldType": "CATEGORICAL" }
    ]
}"""

#We are using the pre-spliting the data into 2 parts, one is used for training the model 
#and the other for the evaluation (test). From more information on splitting data sets, see:
#http://docs.aws.amazon.com/machine-learning/latest/dg/splitting-types.html

#Training data
train_data_split = """{"splitting": {"percentBegin": 0, "percentEnd": 70}}"""
train_data_source_id = resource_pre + 'train-data-titanic-survival'
response = ml.create_data_source_from_s3(
    DataSourceId = train_data_source_id,
    DataSourceName = train_data_source_id,
    DataSpec={
        'DataLocationS3': 's3://' + s3_bucket + "/" + train_data,
        'DataSchema': data_schema,
        'DataRearrangement' : train_data_split
    },
    ComputeStatistics = True
)
#Evaluation data
evaluation_data_split = """{"splitting": {"percentBegin": 71, "percentEnd": 100}}"""
evaluation_data_source_id = resource_pre + 'evaluate-data-titanic-survival'
response = ml.create_data_source_from_s3(
    DataSourceId = evaluation_data_source_id,
    DataSourceName = evaluation_data_source_id,
    DataSpec={
        'DataLocationS3': 's3://' + s3_bucket + "/" + train_data,
        'DataSchema': data_schema,
        'DataRearrangement' : evaluation_data_split
    },
    ComputeStatistics = True
)

print ("Creating datasource and computing stats.. This will take a couple of minutes")
sys.stdout.flush()
waiter = ml.get_waiter('data_source_available')
waiter.wait(FilterVariable='Name', EQ=train_data_source_id, Limit=1)
waiter = ml.get_waiter('data_source_available')
waiter.wait(FilterVariable='Name', EQ=evaluation_data_source_id, Limit=1)
print ("Datasource created!")


In [ ]:
model_id = resource_pre + 'ml-model-titanic-survival'

# Creating the model
response = ml.create_ml_model(
    MLModelId=model_id,
    MLModelName=model_id,
    MLModelType='BINARY',
    TrainingDataSourceId=train_data_source_id
)
print ("Generating model.. This will take a couple of minutes")
sys.stdout.flush()
waiter = ml.get_waiter('ml_model_available')
waiter.wait(FilterVariable='Name',EQ=model_id,Limit=1)
print ("Model created!")


In [ ]:

# Creating an evaluation of the ML model
evaluation_id = resource_pre + 'evaluation-titanic-survival'
response = ml.create_evaluation(
    EvaluationId = evaluation_id,
    EvaluationName = evaluation_id,
    MLModelId = model_id,
    EvaluationDataSourceId = evaluation_data_source_id
)

print ("Evaluating model.. This will take a couple of minutes")
sys.stdout.flush()
waiter = ml.get_waiter('evaluation_available')
waiter.wait(FilterVariable='Name',EQ=evaluation_id,Limit=1)
print ("Evaluation complete!")


In [ ]:
response = ml.describe_evaluations(
    FilterVariable='Name',
    EQ=evaluation_id
)
print ("Out-of-the-box performance (Binary AUC):")
print (response['Results'][0]['PerformanceMetrics']['Properties']['BinaryAUC'])


If you are building a web service that predicts your chances of survival on the titanic, what would you keep a very close eye on?  
(a) true positives  
(b) true negatives  
(c) false positives  
(d) false negatives  

Explore the performance of the model and optionally edit score threshold:

<img align="center" src="https://s3-eu-west-1.amazonaws.com/ipydemo-artifacts/ipython-artifacts-html/1019/explore.png">

In [ ]:
# Enabling real time predictions
for i in range(6):
    response = ml.create_realtime_endpoint(MLModelId=model_id)
    print (response['RealtimeEndpointInfo']['EndpointStatus'])
    if response['RealtimeEndpointInfo']['EndpointStatus'] == 'READY':
        break
    time.sleep(20)
predict_endpoint = response['RealtimeEndpointInfo']['EndpointUrl']
time.sleep(20) #Waiting for endpoint to become active
print ("Realtime prediction endpoint: " + predict_endpoint)


In [ ]:
# Simple predict function
def predict(passenger):
    response = ml.predict(
        MLModelId=model_id,
        Record=passenger,
        PredictEndpoint=predict_endpoint
    )
    print ("Passenger ID: " + passenger['PassengerId'])
    print ("Predicted Label: " + str(response['Prediction']['predictedLabel']))
    print ("Predicted Score: " + str(response['Prediction']['predictedScores']))
    print ("Predictive Model Type: " + str(response['Prediction']['details']['PredictiveModelType']))
    print ("Algorithm: " + str(response['Prediction']['details']['Algorithm']))
    print ("")

passenger1 = {"PassengerId":"1", "Pclass":"3", "Name":"Jack", "Sex":"male","Age":"20","Fare":"10"}
predict(passenger1)

# Let's change the sex to male, but change the class, name, age and fare and add amount of siblings onboard.
passenger2 = {"PassengerId":"3", "Pclass":"1", "Name":"Dr. Jack", "Sex":"male","Age":"60","Fare":"60","SibSp":"3"}
predict(passenger2)
#The predicted score should be much higher.

# Let's change the sex to female and class to first class
passenger3 = {"PassengerId":"2", "Pclass":"1", "Name":"Jacklin", "Sex":"female","Age":"30","Fare":"10"}
predict(passenger3)


Instead of manipulating the values above try out the UI real time predictions:

<img align="center" src="https://s3-eu-west-1.amazonaws.com/ipydemo-artifacts/ipython-artifacts-html/1019/predict.png">

In [ ]:
######################################################################################################
### Clean up environment
######################################################################################################

response = s3.delete_object(Bucket=s3_bucket,Key=train_data)
response = s3.delete_bucket(Bucket=s3_bucket)
print ("Deleted S3 bucket: " + s3_bucket)

response = ml.delete_realtime_endpoint(MLModelId=model_id)
response = ml.delete_evaluation(EvaluationId=evaluation_id)
response = ml.delete_ml_model(MLModelId=model_id)
print ("Deleted Realtime prediction endpoint, evaluation and ML model")

response = ml.delete_data_source(DataSourceId=train_data_source_id)
response = ml.delete_data_source(DataSourceId=evaluation_data_source_id)
print ("Deleted Data sources")
